#transformers

In [ ]:
! git clone https://gitlab.com/bigirqu/quranqa.git

fatal: destination path 'quranqa' already exists and is not an empty directory.


**Install dependices**

In [ ]:
! pip install farasa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 19.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.1 MB/s eta 0:00:00


**Import libraries**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Quran_QA/Notebooks/

ls: cannot access '/content/drive/MyDrive/Quran_QA/Notebooks/': No such file or directory


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
import numpy as np
import json
import sys
from tqdm import tqdm
import time
sys.path.insert(0,"/content/drive/MyDrive/Quran_QA/Notebooks/")
import quranqa22_eval as eval #upload that file


**load data**

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Quran_QA/DataCsv/train.csv")
validation = pd.read_csv("/content/drive/MyDrive/Quran_QA/DataCsv/validation.csv")

In [ ]:
train_dataset_jsonl = []
with open('/content/drive/MyDrive/Quran_QA/json_data/qrcd_v1.1_train.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

with open('/content/drive/MyDrive/Quran_QA/json_data/qrcd_v1.1_dev.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

In [ ]:
len(train_dataset_jsonl)

819

**Prepare data for prediction**

In [ ]:
data = pd.concat([train, validation])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1155 entries, 0 to 162
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pq_id               1155 non-null   object
 1   passage             1155 non-null   object
 2   surah               1155 non-null   int64 
 3   verses              1155 non-null   object
 4   question            1155 non-null   object
 5   answer              1155 non-null   object
 6   start_char          1155 non-null   int64 
 7   processed passage   1155 non-null   object
 8   processed question  1155 non-null   object
 9   processed answer    1155 non-null   object
dtypes: int64(2), object(8)
memory usage: 99.3+ KB


In [ ]:
data = data.filter(items=["pq_id","passage","question"])
data_unique_QA = data.drop_duplicates()

In [ ]:
len(data_unique_QA)

1155

**models**

In [ ]:
from transformers.utils.dummy_pt_objects import RemBertForSequenceClassification
def answer_question(row):
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    result_row = pipe(question=row["question"], context=row["passage"], handle_impossible_answer=True, topk=5)
    return result_row

In [ ]:
def Try_Transformer(tokenizer_name,model_name ):
  result = {}
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  # Apply the function to the "question" and "passage" columns of the DataFrame
  for i in tqdm(range(len(data_unique_QA))):
    result_part = answer_question(data_unique_QA.iloc[i])
    result.update({data_unique_QA.iloc[i]["pq_id"]:result_part})
  return result

In [ ]:
all_scores = {}

*try transformer*

[CAMeL-Lab/bert-base-arabic-camelbert-mix](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-mix)

[CAMeL-Lab/bert-base-arabic-camelbert-ca](https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-ca)

[bert-base-cased](https://huggingface.co/bert-base-cased)

[bert-base-uncased](https://huggingface.co/bert-base-uncased)

[aubmindlab/araelectra-base-generator](https://huggingface.co/aubmindlab/araelectra-base-generator)


[aubmindlab/araelectra-base-discriminator](https://huggingface.co/aubmindlab/araelectra-base-discriminator)

[aubmindlab/bert-large-arabertv2](https://huggingface.co/aubmindlab/bert-large-arabertv2)

1-{'pRR': 0.12906758813551927, 'exact_match': 0.003663003663003663, 'f1': 0.10349880722874749}

2-{'pRR': 0.11646576956143223, 'exact_match': 0.003663003663003663, 'f1': 0.09261998582865365}

3-{'pRR': 0.10786757269906597, 'exact_match': 0.001221001221001221, 'f1': 0.07858916180564851}

4-{'pRR': 0.1003231181462045, 'exact_match': 0.002442002442002442, 'f1': 0.07061552799173902}

5-{'pRR': 0.1336018870627547, 'exact_match': 0.003663003663003663, 'f1': 0.10331842767406314}

6-{'pRR': 0.12716776212524397, 'exact_match': 0.002442002442002442, 'f1': 0.09493013673351615}

7-{'pRR': 0.13128237299724485, 'exact_match': 0.002442002442002442, 'f1': 0.09698476524381845}

In [ ]:
tokenizer = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
model = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:326: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
  0%|          | 1/1155 [00:02<43:37,  2.27s/it]Some weights of BertForQuestionAnswering were not initializ

{'pRR': 0.12906758813551927, 'exact_match': 0.003663003663003663, 'f1': 0.10349880722874749}


In [ ]:
tokenizer = "CAMeL-Lab/bert-base-arabic-camelbert-ca"
model = "CAMeL-Lab/bert-base-arabic-camelbert-ca"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1155 [00:01<33:35,  1.75s/it]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

{'pRR': 0.11646576956143223, 'exact_match': 0.003663003663003663, 'f1': 0.09261998582865365}


In [ ]:
tokenizer = "bert-base-cased"
model = "bert-base-cased"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1155 [00:01<34:39,  1.80s/it]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 2/1155 [00:03<33:36,  1.75s/it]Some w

{'pRR': 0.10786757269906597, 'exact_match': 0.001221001221001221, 'f1': 0.07858916180564851}


In [ ]:
tokenizer = "bert-base-uncased"
model = "bert-base-uncased"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1155 [00:03<58:27,  3.04s/it]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 2/1155 [00:05<52:44,  2.74s/it]

{'pRR': 0.1003231181462045, 'exact_match': 0.002442002442002442, 'f1': 0.07061552799173902}


In [ ]:
tokenizer = "aubmindlab/araelectra-base-generator"
model = "aubmindlab/araelectra-base-generator"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-generator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-generator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1155 [00:01<25:09,  1.31s/it]Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-generator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

{'pRR': 0.1336018870627547, 'exact_match': 0.003663003663003663, 'f1': 0.10331842767406314}


In [ ]:
tokenizer = "aubmindlab/araelectra-base-discriminator"
model = "aubmindlab/araelectra-base-discriminator"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/1155 [00:03<1:01:46,  3.21s/it]Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/araelectra-base-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it

{'pRR': 0.12716776212524397, 'exact_match': 0.002442002442002442, 'f1': 0.09493013673351615}


In [ ]:
tokenizer = "aubmindlab/bert-large-arabertv2"
model = "aubmindlab/bert-large-arabertv2"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-large-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/1155 [00:00<?, ?it/s]Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-large-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:326: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
  0%|          | 1/1155 [00:06<2:02:16,  6.36s/it]Some weights of BertForQuestionAnswering were not initialized from the mode

{'pRR': 0.13128237299724485, 'exact_match': 0.002442002442002442, 'f1': 0.09698476524381845}


In [19]:
import json

# Specify the file name and path
filename = 'all_scores.json'

 # Write the scores dictionary to a JSON file
with open(filename, 'w') as json_file:
  json.dump(all_scores, json_file)

In [20]:
scores=pd.read_json("all_scores.json").T
scores.head()

,pRR,exact_match,f1
CAMeL-Lab/bert-base-arabic-camelbert-mix,0.129068,0.003663,0.103499
CAMeL-Lab/bert-base-arabic-camelbert-ca,0.116466,0.003663,0.092620
bert-base-cased,0.107868,0.001221,0.078589
bert-base-uncased,0.100323,0.002442,0.070616
aubmindlab/araelectra-base-generator,0.133602,0.003663,0.103318
